# Задание №4 Изучение OLS, Lasso и Random Forest в задаче регрессии 

In [12]:
import warnings
warnings.filterwarnings("ignore")

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import Lasso, LassoCV, LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import (GridSearchCV, cross_val_score, train_test_split)
from sklearn.preprocessing import StandardScaler

In [13]:
DATA_PATH = "https://raw.githubusercontent.com/Yorko/mlcourse.ai/main/data/"
data = pd.read_csv(DATA_PATH + "winequality-white.csv", sep=";")

In [16]:
data.head()

,fixed acidity,volatile acidity,citric acid,residual sugar,chlorides,free sulfur dioxide,total sulfur dioxide,density,pH,sulphates,alcohol,quality
0,7.0,0.27,0.36,20.7,0.045,45.0,170.0,1.0010,3.00,0.45,8.8,6
1,6.3,0.30,0.34,1.6,0.049,14.0,132.0,0.9940,3.30,0.49,9.5,6
2,8.1,0.28,0.40,6.9,0.050,30.0,97.0,0.9951,3.26,0.44,10.1,6
3,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6
4,7.2,0.23,0.32,8.5,0.058,47.0,186.0,0.9956,3.19,0.40,9.9,6


In [18]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4898 entries, 0 to 4897
Data columns (total 12 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed acidity         4898 non-null   float64
 1   volatile acidity      4898 non-null   float64
 2   citric acid           4898 non-null   float64
 3   residual sugar        4898 non-null   float64
 4   chlorides             4898 non-null   float64
 5   free sulfur dioxide   4898 non-null   float64
 6   total sulfur dioxide  4898 non-null   float64
 7   density               4898 non-null   float64
 8   pH                    4898 non-null   float64
 9   sulphates             4898 non-null   float64
 10  alcohol               4898 non-null   float64
 11  quality               4898 non-null   int64  
dtypes: float64(11), int64(1)
memory usage: 459.3 KB


### Задание 1. Отделите целевой признак, разделите данные в пропорции 7:3 (30% формируют контрольный набор, используйте random_state=17) и выполните предварительную обработку данных с помощью StandardScaler.

In [20]:
y = data["quality"] 
X = data.drop("quality", axis=1)

#### Разделим данные так, как указано в задании.

In [23]:
X_train, X_holdout, y_train, y_holdout = train_test_split(X, y, test_size=0.3, random_state=17)

#### Скалируем наши данные, избавляясь от выбросов.

In [26]:
scaler = StandardScaler()

In [28]:
X_train_scaled = scaler.fit_transform(X_train)
X_holdout_scaled = scaler.transform(X_holdout)

## Линейная регрессия

In [27]:
linreg = LinearRegression()
linreg.fit(X_train_scaled, y_train);

#### Теперь у нас есть настоенная линейная регрессия. Ответим с  её помощью на следующие вопросы:

### Вопрос 1. Каковы средние квадратические ошибки прогнозов модели на обучающих и контрольных наборах?

In [35]:
print("Mean squared error (train): %.3f" % mean_squared_error(y_train, linreg.predict(X_train_scaled)))
print("Mean squared error (test): %.3f" % mean_squared_error(y_holdout, linreg.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.584


### Вопрос 2. Какой параметр эта линейная регрессионная модель рассматривает как наиболее сильно влияющий на качество вина?

In [53]:
lin_coefs = pd.DataFrame({"coefs": linreg.coef_, "coefs_abs": np.abs(linreg.coef_)},
index=data.columns.drop("quality"), )
lin_coefs.sort_values(by="coefs_abs", ascending=False)

,coefs,coefs_abs
density,-0.665720,0.665720
residual sugar,0.538164,0.538164
volatile acidity,-0.192260,0.192260
pH,0.150036,0.150036
alcohol,0.129533,0.129533
fixed acidity,0.097822,0.097822
sulphates,0.062053,0.062053
free sulfur dioxide,0.042180,0.042180
total sulfur dioxide,0.014304,0.014304
chlorides,0.008127,0.008127


#### Ответ: density (плотность/густота).

## Лассо-регрессия

### Обучите модель LASSO с помощью α = 0.01  (слабая регуляризация) и масштабированные данные. Снова устанавливаем random_state=17.

In [59]:
lasso = Lasso(alpha=0.01, random_state=17)
lasso.fit(X_train_scaled, y_train)

Lasso(alpha=0.01, random_state=17)

### Задание 2. Какая характеристика наименее информативна при прогнозировании качества вина согласно этой модели LASSO?

In [71]:
lasso_coefs = pd.DataFrame({"coefs": lasso.coef_, "coefs_abs": np.abs(lasso.coef_)},
index=data.columns.drop("quality"), )
lasso_coefs.sort_values(by="coefs_abs", ascending=False)

,coefs,coefs_abs
alcohol,0.322425,0.322425
residual sugar,0.256363,0.256363
density,-0.235492,0.235492
volatile acidity,-0.188479,0.188479
pH,0.067277,0.067277
free sulfur dioxide,0.043088,0.043088
sulphates,0.029722,0.029722
chlorides,-0.002747,0.002747
fixed acidity,-0.000000,0.000000
citric acid,-0.000000,0.000000


#### Как мы видим, это total sulfur dioxide (общий диоксид серы), citric acid (лимонная кислота) и fixed acidity (фиксированная кислотность).

### Задание 3. Обучите LassoCV с random_state=17, чтобы выбрать лучшее значение α в 5-кратной перекрестной проверке.

In [32]:
alph = np.logspace(-6, 2, 200)
lasso_cv = LassoCV(random_state=17, cv=5, alphas=alph)
lasso_cv.fit(X_train_scaled, y_train)

LassoCV(alphas=array([1.00000000e-06, 1.09698580e-06, 1.20337784e-06, 1.32008840e-06,
       1.44811823e-06, 1.58856513e-06, 1.74263339e-06, 1.91164408e-06,
       2.09704640e-06, 2.30043012e-06, 2.52353917e-06, 2.76828663e-06,
       3.03677112e-06, 3.33129479e-06, 3.65438307e-06, 4.00880633e-06,
       4.39760361e-06, 4.82410870e-06, 5.29197874e-06, 5.80522552e-06,
       6.36824994e-06, 6.98587975e-0...
       1.18953407e+01, 1.30490198e+01, 1.43145894e+01, 1.57029012e+01,
       1.72258597e+01, 1.88965234e+01, 2.07292178e+01, 2.27396575e+01,
       2.49450814e+01, 2.73644000e+01, 3.00183581e+01, 3.29297126e+01,
       3.61234270e+01, 3.96268864e+01, 4.34701316e+01, 4.76861170e+01,
       5.23109931e+01, 5.73844165e+01, 6.29498899e+01, 6.90551352e+01,
       7.57525026e+01, 8.30994195e+01, 9.11588830e+01, 1.00000000e+02]),
        cv=5, random_state=17)

#### Лучшее значение α:

In [34]:
lasso_cv.alpha_

0.0002833096101839324

### Вопрос 3. Какая характеристика наименее информативна при прогнозировании качества вина согласно настроенной модели LASSO?

In [37]:
lasso_coefs_tuned = pd.DataFrame({"coefs": lasso_cv.coef_, "coefs_abs": np.abs(lasso_cv.coef_)},
index=data.columns.drop("quality"), )
lasso_coefs_tuned.sort_values(by="coefs_abs", ascending=False)

,coefs,coefs_abs
density,-0.648161,0.648161
residual sugar,0.526883,0.526883
volatile acidity,-0.192049,0.192049
pH,0.146549,0.146549
alcohol,0.137115,0.137115
fixed acidity,0.093295,0.093295
sulphates,0.060939,0.060939
free sulfur dioxide,0.042698,0.042698
total sulfur dioxide,0.012969,0.012969
chlorides,0.006933,0.006933


#### Ответ: citric acid (лимонная кислота).

### Вопрос 4. Каковы средние квадратические ошибки настроенных прогнозов LASSO на обучающих и контрольных наборах?


In [94]:
print("Mean squared error (train): %.3f"
    % mean_squared_error(y_train, lasso_cv.predict(X_train_scaled)))
print("Mean squared error (test): %.3f"
    % mean_squared_error(y_holdout, lasso_cv.predict(X_holdout_scaled)))

Mean squared error (train): 0.558
Mean squared error (test): 0.583


#### Ответ: 0.558 и 0.583 соответственно.

## Случайный лес

### Задание 4. Обучите случайный лес с готовыми параметрами, установив только random_state равным 17.

In [104]:
r_forest = RandomForestRegressor(random_state=17)
r_forest.fit(X_train_scaled, y_train)

RandomForestRegressor(random_state=17)

### Вопрос 5. Каковы средние квадратические ошибки модели RF на обучающем наборе, при перекрестной проверке (cross_val_score с scoring='neg_mean_squared_error' и другими аргументами, для которых оставлены значения по умолчанию) и на контрольном наборе?

In [112]:
print( "Mean squared error (train): %.3f"
    % mean_squared_error(y_train, r_forest.predict(X_train_scaled)))
print( "Mean squared error (cv): %.3f"
    % np.mean(np.abs(cross_val_score(r_forest, X_train_scaled, y_train, scoring="neg_mean_squared_error"))))
print( "Mean squared error (test): %.3f"
    % mean_squared_error(y_holdout, r_forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.053
Mean squared error (cv): 0.414
Mean squared error (test): 0.372


#### Теперь настроим гиперпараметры - количество рассматриваемых свойств и максимальню глубину деревьев нашего случайного леса. После снова проверим среднеквадратичную ошибку перекрестной проверки и среднеквадратичную ошибку на контрольном наборе.

In [116]:
forest_hyperpar = {"max_depth": list(range(10, 25)), "max_features": list(range(6, 12))}
tuned_forest = GridSearchCV(RandomForestRegressor(n_jobs=-1, random_state=17),forest_hyperpar,
    scoring="neg_mean_squared_error",n_jobs=-1,cv=5,verbose=True,)
tuned_forest.fit(X_train_scaled, y_train)

Fitting 5 folds for each of 90 candidates, totalling 450 fits


GridSearchCV(cv=5, estimator=RandomForestRegressor(n_jobs=-1, random_state=17),
             n_jobs=-1,
             param_grid={'max_depth': [10, 11, 12, 13, 14, 15, 16, 17, 18, 19,
                                       20, 21, 22, 23, 24],
                         'max_features': [6, 7, 8, 9, 10, 11]},
             scoring='neg_mean_squared_error', verbose=True)

In [120]:
tuned_forest.best_params_, tuned_forest.best_score_

({'max_depth': 21, 'max_features': 6}, -0.39773288191505934)

In [122]:
print( "Mean squared error (train): %.3f"
    % mean_squared_error(y_train, tuned_forest.predict(X_train_scaled)))
print( "Mean squared error (cv): %.3f"
    % np.mean(np.abs(cross_val_score(tuned_forest, X_train_scaled, y_train, scoring="neg_mean_squared_error"))))
print( "Mean squared error (test): %.3f"
    % mean_squared_error(y_holdout, tuned_forest.predict(X_holdout_scaled)))

Mean squared error (train): 0.054
Fitting 5 folds for each of 90 candidates, totalling 450 fits
Fitting 5 folds for each of 90 candidates, totalling 450 fits
Fitting 5 folds for each of 90 candidates, totalling 450 fits
Fitting 5 folds for each of 90 candidates, totalling 450 fits
Fitting 5 folds for each of 90 candidates, totalling 450 fits
Mean squared error (cv): 0.402
Mean squared error (test): 0.366


#### Ответ: 0.053 (train) , 0.414 (cv), 0.372 (test) для ненастроенного леса и 0.054 (train), 0.402 (cv), 0.366 (test) для настроенного леса. Видно явное улучшение результатов после настройки гиперпараметров леса.

### Вопрос 6. Каковы средние квадратические ошибки настроенной модели RF при перекрестной проверке (cross_val_score с scoring='neg_mean_squared_error' и другими аргументами, для которых оставлены значения по умолчанию) и на контрольном наборе?

In [132]:
print("Mean squared error (cv): %.3f"
    % np.mean(np.abs(cross_val_score(tuned_forest.best_estimator_, X_train_scaled,y_train,
                scoring="neg_mean_squared_error",))))
print("Mean squared error (test): %.3f"
    % mean_squared_error(y_holdout, tuned_forest.predict(X_holdout_scaled)))

Mean squared error (cv): 0.398
Mean squared error (test): 0.366


#### Ответ: 0.398 и 0.366 соответственно.

### Вопрос 7: Какая самая важная особенность, согласно модели Random Forest?

In [137]:
rf_importance = pd.DataFrame(
    tuned_forest.best_estimator_.feature_importances_,
    columns=["coef"],
    index=data.columns[:-1],
)
rf_importance.sort_values(by="coef", ascending=False)

,coef
alcohol,0.206056
volatile acidity,0.117578
free sulfur dioxide,0.111556
density,0.088549
pH,0.073659
total sulfur dioxide,0.073640
chlorides,0.073366
residual sugar,0.072072
citric acid,0.062601
fixed acidity,0.061813


#### Ответ: alcohol (алкоголь).